In [1]:
file_path = (
    "Resume_11_2024.pdf"
)

In [2]:
from PyPDF2 import PdfReader

In [3]:
reader = PdfReader(file_path)
print(len(reader.pages))
text = ""
for page in reader.pages:
    text += page.extract_text()
# page = reader.pages[0]
# text = page.extract_text()


2


In [4]:
print(text)

Seyhan Emre Gorucu
Calgary, AB, Canada
(403) 827-4222 | segorucu@gmail.com ||| Canadian citizen & US Greencard holder
WorkExperience
ComputerModellingGroup Calgary,AB
SCIENTIFIC SOFTWARE DEVELOPMENT ENGINEER / SENIOR SIMULATION SCIENTIST Aug2013-Present
•Workedondevelopment,maintenanceandsupportofGEM(Fortranbasedequation-of-statecompositionalsimulatorusedforthesimulation
ofoil-gas-waterflow).
•Developedand/orimprovedfeaturesforsimulationofaqueousreactions,mineralreactions,surfacecomplexation,pitzeractivitymodel,phase
behavior,finesandproppants,single-componentisenthalpicflashcalculations.
•Thesefeaturesareusedbyhundredsofenergycompaniesinordertoimprovedecisionmakingforreservoirmanagement.
•ConsultingandSupport.
ThePennsylvaniaStateUniversity StateCollege,PA
RESEARCH ASSISTANT Aug. 2010-Apr. 2013
•Introduced new algorithms that compute the number of physical phases and chemical compositions in an oil and gas reservoir. As these
calculationsarerepeatedbillionsoftimesduringanumericalr

In [5]:
import openai
import os
from openai import OpenAI
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
openai_client = OpenAI(api_key=OPENAI_API_KEY)

In [6]:
prompt = f"""Given the following ocr result from a resume, put it in a readable format.
    OCR result
    {text}
"""

In [7]:
chat_response = openai_client.chat.completions.create(
        model="gpt-4o",  # Specify the GPT-4 model
        messages=[
            {
                "role": "user",
                "content": prompt
            }
        ],
    )

In [8]:
resume_clean = chat_response.choices[0].message.content

In [9]:
print(resume_clean)

**Seyhan Emre Gorucu**  
Calgary, AB, Canada  
Phone: (403) 827-4222 | Email: segorucu@gmail.com  
Canadian Citizen & US Green Card Holder  

---

**Work Experience**

**Computer Modelling Group**  
Calgary, AB  
**Scientific Software Development Engineer / Senior Simulation Scientist**  
August 2013 - Present  
- Worked on development, maintenance, and support of GEM (Fortran-based equation-of-state compositional simulator used for the simulation of oil-gas-water flow).
- Developed and/or improved features for simulation of aqueous reactions, mineral reactions, surface complexation, Pitzer activity model, phase behavior, fines and proppants, single-component isenthalpic flash calculations.
- These features are used by hundreds of energy companies to improve decision-making for reservoir management.
- Provided consulting and support.

**The Pennsylvania State University**  
State College, PA  
**Research Assistant**  
August 2010 - April 2013  
- Introduced new algorithms that compute 

In [10]:
from elasticsearch import Elasticsearch

In [11]:
es_client = Elasticsearch(
        ['https://localhost:9200'],
        basic_auth=('elastic', 'elastic'),
        verify_certs = False
    )

C:\Users\segor\miniconda3\envs\brave\Lib\site-packages\elasticsearch\_sync\client\__init__.py:402: SecurityWarning: Connecting to 'https://localhost:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(


In [29]:
def get_matching_jobs(index_name, location, position):
    query = {
        "query": {
            "bool": {
                "must": [
                    {"match": {"location": location}},
                    {"match": {"query": position}}
                ]
            }
        }
    }
    results = es_client.search(index=index_name, body=query, size=1000)
    return results['hits']['hits']


In [30]:
jobs = get_matching_jobs("brave_project","Toronto", "Machine Learning Engineer")

C:\Users\segor\AppData\Local\Temp\ipykernel_25752\1041610920.py:12: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  results = es_client.search(index=index_name, body=query, size=1000)
C:\Users\segor\miniconda3\envs\brave\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [31]:
texts = [f"job id: {i} Company name: {job['_source']['companyName']} Title: {job['_source']['title']} description: {job['_source']['description']}" for i, job in enumerate(jobs)]

In [32]:
texts[0]

"job id: 0 Company name: Altus Group Title: Machine Learning Engineer (Remote) description: Job Category:\nTechnology\n\nPay Grade Range:\n$66,605.00 - $155,405.00\n\nDisclaimer: The base salary range represents the low and high end of Altus Group’s “Pay Grade Range” for this position in the primary work location. Actual hiring salaries will vary depending on factors including but not limited to work experience, and geographic market data for the role. The Pay Grade Range listed above does not reflect Altus Group’s total compensation for employees. Other rewards may include an annual bonus, flexible work arrangements, and region-specific benefits.\n\nUnlock your Altus Experience!\n\nIf you’re looking to advance your career in data analytics, expertise, and technology for the rapidly growing global CRE market, there’s no better place than Altus Group. At Altus, our work is purposeful. Every day, our employees drive impact, innovate, and shape the global commercial real estate (CRE) and 

In [24]:
from langchain_openai import ChatOpenAI
from langchain_chroma import Chroma
from langchain import hub
from typing_extensions import List, TypedDict
from langchain_core.documents import Document
from langgraph.graph import START, StateGraph
from langchain.embeddings import OpenAIEmbeddings
llm = ChatOpenAI(model="gpt-4o")
embeddings = OpenAIEmbeddings()
vector_store = Chroma.from_texts(texts, embeddings)
prompt = hub.pull("rlm/rag-prompt")

In [25]:
# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}

graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [26]:
question = f"Given the following resume: {resume_clean} and list of jobs {texts}, analyze which jobs match the resume better. Return the ids, company names, job titles and summaries of the 3 best matching jobs."

In [27]:
response = graph.invoke({"question": question})

In [28]:
print(response["answer"])

1. **Job ID: 0, Company Name: Altus Group, Job Title: Machine Learning Engineer (Remote)**  
   Summary: Altus Group is seeking a Machine Learning Engineer to support Altus Labs, focused on developing and managing scalable data pipelines and machine learning models to drive better client investment and operating outcomes in the commercial real estate market. The role involves collaboration with Data Scientists, Analysts, and DevOps Engineers, emphasizing MLOps strategies, model development, deployment, and performance optimization.

2. **Job ID: 7, Company Name: Gusto, Job Title: Machine Learning Engineer, Payments & Risk**  
   Summary: Gusto is looking for a technical leader to drive machine learning and AI in the payment and risk domains, building a model-driven risk platform and collaborating with cross-functional teams to enhance Gusto's products. The role requires deploying predictive models, developing scalable frameworks, and leveraging data to improve risk management, with a f